# COMP47670 - Assignment 2 - Text Classification
**Student Name: Meleesha Mayola Dsouza, Nikil Mohan**<br>
**Student Number: 18200024, 18200037**

In [70]:
# Import the required libraries
import os
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
%matplotlib inline

## Task 1: Selecting the review categories and scraping the data

For the purpose of this assignment, we have chosen the following categories: 
1. Health and Medical
2. Automotive


In [71]:
# For specifying the website, we append the individual webpages to the base website URL
endpoint = 'http://mlg.ucd.ie/modules/yalp/'
# The categories chosen have been represented in the form of a list
categories = ['health_medical_list.html', 'automotive_list.html'] 

# The extract_page function is created to extract the overall data present in the webpage
def extract_page(url):
    response = requests.get(url)
    data = response.text
    # We use the BeautifulSoup package to parse the data that is collected for webscraping
    soup = BeautifulSoup(data,'html.parser')
    return soup

# The extract_reviews function is created to extract the reviews from the webpoge
def extract_reviews(url):
    soup = extract_page(url)
    # We use the 'div' as the identifier as all the reviews are present within this tag
    reviews_block = soup.findAll("div", { "class" : "review" })
    # We create a list called review_list to store all the reviews that are present in the category
    review_list =[]
    for reviews in reviews_block:
        review = {}
        # We use the 'img' as the identifier as all the ratings are present within this tag
        star = reviews.find('img')
        review["comments"] = reviews.find("p", { "class" : "text" }).get_text()
        # We use the 'alt' attribute as the identifier as the rating present in this attribute can be easily obtained
        # We use the concept of any rating having a value of 4 or 5 as Positive
        # We use the concept of any rating having a value of 1, 2 or 3 as Negative
        review["rating"] = 'positive' if int(star.get('alt').split('-')[0]) >= 4 else 'negative'
        review_list.append(review)        
    return review_list

# The extract_data function is created to extract the data from each of the review links
# This function in turn calls the extract_page and the extract_reviews
def extract_data(url):
    soup = extract_page(url)
    links = soup.find_all('a')
    review_list =[]
    for link in links:
        # We identify the links of each of the reviews by extracting the values present in the href attribute
        url = endpoint + link.get('href')
        review = extract_reviews(url)
        # We concatenate the reviews from all the links
        review_list = review_list+review
    return review_list

# The create_database function is created to write the reviews and ratings to csv files for easier processing
# The function will be called each time for each category
def create_database(category_name, reviews):
    header = reviews[0].keys()
    with open(category_name, 'w',newline='', encoding="utf-8") as output_file:
        dict_writer = csv.DictWriter(output_file, header)
        dict_writer.writeheader()
        dict_writer.writerows(reviews)


Create database for category A, Health and medical reviews

In [72]:
# We access the page for the reviews of the category 'Health and Medical'
# We scrape the data using the functions defined by us
# This data is stored in separate files in the csv format
url = endpoint + categories[0]
review_list = extract_data(url)
create_database('Health_Medical.csv', review_list)

Create database for category B, Automotive reviews

In [73]:
# We access the page for the reviews of the category 'Automotive'
# We scrape the data using the functions defined by us
# This data is stored in separate files in the csv format
url = endpoint + categories[1]
review_list = extract_data(url)
create_database('Automotive.csv', review_list)

In [74]:
# The dataframe dataset_A is a dataframe that has the data obtained from csv
dataset_A = pd.read_csv('Health_Medical.csv')
# We check the dimensions of the data that we have
print(dataset_A.shape)
dataset_A.head()

(1450, 2)


,comments,rating
0,I have so many good things to say about this p...,positive
1,I found them to be highly skilled and an exper...,positive
2,Where do I even begin? This office has been so...,positive
3,I went in because I had toothache and needed a...,positive
4,Found a new dental office. This place is amazi...,positive


In [75]:
# The dataframe dataset_B is a dataframe that has the data obtained from csv
dataset_B = pd.read_csv('Automotive.csv')
# We check the dimensions of the data that we have
print(dataset_B.shape)
dataset_B.head()

(1455, 2)


,comments,rating
0,I arrived at 3 PM and the dealership closed at...,positive
1,I dropped my car off on a Wednesday morning fo...,negative
2,My parents have been buying cars off of Donna ...,positive
3,I recently bought another car from Donna Dunni...,positive
4,I had to schedule an appointment due to the ai...,positive


In [102]:
# The preprocess_text function is called to handle the preprocessing steps
# These steps are performed before we try to classify the data
    def preprocess_text(text):
    # The data that is passed as the input is first converted to lowercase to standardize it
    # We then tokenize this data into individual tokens
    tokenized_words = word_tokenize(text.lower())
    # We remove the words which are not alpha-numeric like punctuations
    normalised_words = [word for word in tokenized_words if word.isalnum()]
    # We count the number of occurence of each word in the corpus
    fdist = nltk.FreqDist(normalised_words)
    print(fdist.most_common(50))
    # We remove the stopwords in the corpus like 'is', 'an', etc. as they do not contribute to the efficiency of the classifier
    return ' '.join([word for word in normalised_words if word.lower() not in stopwords.words('english')])

In [103]:
def preprocess_reviews(dataset):
    for indx, comments in enumerate(dataset['comments']):
        dataset['comments'][indx] = preprocess_text(comments)

In [104]:
preprocess_reviews(dataset_A)
print()

[('cleaning', 3), ('good', 2), ('filling', 2), ('dental', 2), ('many', 1), ('things', 1), ('say', 1), ('place', 1), ('needed', 1), ('decided', 1), ('part', 1), ('dreading', 1), ('carlos', 1), ('magic', 1), ('touch', 1), ('felt', 1), ('pain', 1), ('process', 1), ('way', 1), ('faster', 1), ('thought', 1), ('would', 1), ('next', 1), ('came', 1), ('mandap', 1), ('friendly', 1), ('patient', 1), ('took', 1), ('time', 1), ('answer', 1), ('questions', 1), ('teeth', 1), ('gums', 1), ('etc', 1), ('gave', 1), ('lot', 1), ('tips', 1), ('personal', 1), ('care', 1), ('office', 1), ('cool', 1), ('futuristic', 1), ('look', 1), ('well', 1), ('receptionist', 1), ('always', 1), ('accommodating', 1), ('glad', 1), ('finally', 1), ('found', 1)]
[('care', 2), ('dental', 2), ('like', 2), ('real', 2), ('found', 1), ('highly', 1), ('skilled', 1), ('experienced', 1), ('dentistry', 1), ('willing', 1), ('work', 1), ('put', 1), ('together', 1), ('reasonable', 1), ('plan', 1), ('take', 1), ('needs', 1), ('without', 

[('would', 3), ('looked', 2), ('ordering', 2), ('wrong', 2), ('item', 2), ('asked', 2), ('told', 2), ('shipping', 2), ('also', 2), ('cost', 2), ('product', 2), ('reviews', 1), ('company', 1), ('ordered', 1), ('worded', 1), ('properly', 1), ('website', 1), ('receiving', 1), ('return', 1), ('charged', 1), ('ways', 1), ('items', 1), ('sold', 1), ('noticed', 1), ('correct', 1), ('since', 1), ('strictly', 1), ('fault', 1), ('anything', 1), ('said', 1), ('keep', 1), ('order', 1), ('right', 1), ('one', 1), ('please', 1), ('talk', 1), ('supervisor', 1), ('ask', 1), ('could', 1), ('waive', 1), ('never', 1), ('received', 1), ('response', 1), ('poor', 1), ('customer', 1), ('service', 1), ('found', 1), ('exact', 1), ('less', 1), ('several', 1)]
[('weeks', 3), ('customer', 3), ('service', 3), ('buy', 2), ('company', 2), ('order', 2), ('wait', 2), ('might', 2), ('never', 1), ('arrived', 1), ('spent', 1), ('since', 1), ('arguing', 1), ('tell', 1), ('claim', 1), ('filed', 1), ('post', 1), ('office', 1

[('foot', 2), ('massage', 2), ('lily', 2), ('went', 1), ('finishing', 1), ('four', 1), ('day', 1), ('tradeshow', 1), ('lot', 1), ('standing', 1), ('referred', 1), ('see', 1), ('right', 1), ('amazing', 1), ('highly', 1), ('recommend', 1), ('visited', 1), ('places', 1), ('las', 1), ('vegas', 1), ('yet', 1), ('go', 1), ('bad', 1), ('best', 1), ('far', 1)]
[('ask', 1), ('tom', 1), ('got', 1), ('foot', 1), ('calf', 1), ('massage', 1), ('w', 1), ('reflexology', 1), ('finished', 1), ('tough', 1), ('work', 1), ('needed', 1), ('relief', 1), ('took', 1), ('right', 1), ('away', 1), ('amazing', 1), ('minutes', 1), ('get', 1), ('lot', 1), ('massages', 1), ('well', 1), ('worth', 1)]
[('massage', 5), ('got', 2), ('giving', 2), ('hear', 2), ('foot', 1), ('yesterday', 1), ('min', 1), ('minutes', 1), ('thought', 1), ('pricy', 1), ('running', 1), ('half', 1), ('marathon', 1), ('desperate', 1), ('soon', 1), ('walked', 1), ('lady', 1), ('tried', 1), ('rush', 1), ('back', 1), ('without', 1), ('discussing', 

[('always', 1), ('great', 1), ('experience', 1), ('staff', 1), ('really', 1), ('nice', 1), ('jamal', 1), ('best', 1), ('recommend', 1), ('place', 1), ('friends', 1), ('area', 1)]
[('great', 2), ('really', 1), ('experience', 1), ('broke', 1), ('tooth', 1), ('friday', 1), ('holiday', 1), ('closed', 1), ('sent', 1), ('email', 1), ('anyway', 1), ('response', 1), ('appointment', 1), ('within', 1), ('hour', 1), ('saturday', 1), ('noon', 1), ('doc', 1), ('nice', 1), ('space', 1), ('new', 1), ('gear', 1)]
[('best', 2), ('dentist', 2), ('staff', 2), ('kind', 2), ('rest', 2), ('pain', 2), ('bite', 2), ('place', 2), ('experience', 1), ('ever', 1), ('greeted', 1), ('pleasant', 1), ('front', 1), ('desk', 1), ('wait', 1), ('minutes', 1), ('getting', 1), ('chair', 1), ('starting', 1), ('initial', 1), ('exam', 1), ('returned', 1), ('weeks', 1), ('later', 1), ('get', 1), ('fillings', 1), ('zunaid', 1), ('jamal', 1), ('incredibly', 1), ('upbeat', 1), ('person', 1), ('incredible', 1), ('work', 1), ('alon

[('place', 2), ('great', 1), ('go', 1), ('always', 1), ('ppl', 1), ('coming', 1), ('dispensary', 1), ('work', 1), ('asking', 1), ('carry', 1), ('sap', 1), ('good', 1), ('thing', 1), ('let', 1), ('forget', 1), ('first', 1)]
[('specialist', 3), ('free', 3), ('new', 2), ('patient', 2), ('look', 2), ('looking', 2), ('gram', 2), ('staff', 1), ('environment', 1), ('az', 1), ('organix', 1), ('great', 1), ('sign', 1), ('quick', 1), ('answer', 1), ('questions', 1), ('id', 1), ('told', 1), ('brought', 1), ('flower', 1), ('strains', 1), ('applied', 1), ('allowed', 1), ('smell', 1), ('special', 1), ('advertised', 1), ('really', 1), ('got', 1), ('almost', 1), ('like', 1), ('getting', 1), ('grams', 1), ('let', 1), ('take', 1), ('grown', 1), ('papa', 1), ('og', 1), ('top', 1), ('shelf', 1), ('one', 1), ('well', 1), ('seemed', 1), ('excited', 1), ('available', 1), ('class', 1), ('strain', 1)]
[('place', 3), ('first', 1), ('seemed', 1), ('bit', 1), ('scarey', 1), ('since', 1), ('back', 1), ('building',

...

[('medical', 2), ('family', 1), ('part', 1), ('university', 1), ('sports', 1), ('medicine', 1), ('many', 1), ('years', 1), ('doctor', 1), ('pa', 1), ('staff', 1), ('always', 1), ('helpful', 1), ('matter', 1), ('issues', 1), ('gone', 1), ('listen', 1), ('needs', 1), ('combine', 1), ('expertise', 1), ('deliver', 1), ('best', 1), ('care', 1)]
[('patient', 2), ('like', 2), ('university', 1), ('sports', 1), ('medicine', 1), ('since', 1), ('office', 1), ('pointe', 1), ('mccoy', 1), ('anastasi', 1), ('speak', 1), ('intelligent', 1), ('grown', 1), ('missing', 1), ('attitude', 1), ('comes', 1), ('across', 1), ('doctors', 1), ('always', 1), ('felt', 1), ('genuinely', 1), ('care', 1), ('health', 1), ('issues', 1), ('typically', 1), ('seen', 1), ('within', 1), ('minutes', 1), ('appointment', 1), ('time', 1), ('rare', 1), ('days', 1), ('never', 1), ('seem', 1), ('rushed', 1), ('move', 1), ('next', 1), ('could', 1), ('happier', 1), ('practice', 1)]
[('anastasi', 3), ('could', 2), ('made', 2), ('staf

[('concerns', 1), ('previous', 1), ('reviews', 1), ('addressed', 1), ('improved', 1), ('time', 1), ('way', 1), ('make', 1), ('appointments', 1), ('online', 1)]
[('care', 6), ('team', 5), ('health', 4), ('reviews', 3), ('medical', 3), ('doctor', 3), ('get', 3), ('time', 3), ('village', 3), ('reading', 2), ('years', 2), ('modern', 2), ('rude', 2), ('front', 2), ('desk', 2), ('family', 2), ('king', 2), ('feel', 2), ('within', 2), ('staff', 2), ('people', 2), ('would', 2), ('little', 2), ('apt', 2), ('one', 2), ('yes', 2), ('liberty', 2), ('know', 2), ('also', 2), ('design', 2), ('thought', 1), ('post', 1), ('became', 1), ('patient', 1), ('clinic', 1), ('ago', 1), ('start', 1), ('mumbling', 1), ('decor', 1), ('friendly', 1), ('smiles', 1), ('candies', 1), ('let', 1), ('cut', 1), ('chase', 1), ('outstanding', 1), ('practitioner', 1), ('charles', 1), ('truly', 1), ('genuinely', 1)]
[('great', 2), ('medical', 2), ('team', 1), ('coming', 1), ('guys', 1), ('years', 1), ('provided', 1), ('suppor

[('roy', 3), ('glasses', 3), ('went', 2), ('recently', 1), ('visionworks', 1), ('mayo', 1), ('blvd', 1), ('spoke', 1), ('eye', 1), ('wrong', 1), ('prescription', 1), ('due', 1), ('miscommunication', 1), ('another', 1), ('provider', 1), ('really', 1), ('way', 1), ('help', 1), ('get', 1), ('worked', 1), ('could', 1), ('nicer', 1), ('strongly', 1), ('recommend', 1), ('store', 1), ('manager', 1)]
[('friendly', 2), ('find', 2), ('staff', 2), ('look', 2), ('thank', 2), ('nervous', 1), ('getting', 1), ('eye', 1), ('checked', 1), ('years', 1), ('quick', 1), ('explained', 1), ('whole', 1), ('process', 1), ('helped', 1), ('make', 1), ('comfortable', 1), ('could', 1), ('perfect', 1), ('prescription', 1), ('helpful', 1), ('pamela', 1), ('assisted', 1), ('scoring', 1), ('killer', 1), ('deals', 1), ('months', 1), ('contacts', 1), ('new', 1), ('glasses', 1), ('sunglasses', 1), ('pam', 1), ('rest', 1), ('also', 1), ('real', 1), ('refreshing', 1), ('would', 1), ('suggest', 1), ('frame', 1), ('colors', 

[('like', 3), ('feel', 2), ('rude', 2), ('filling', 1), ('paperwork', 1), ('lots', 1), ('far', 1), ('receptionist', 1), ('tone', 1), ('friendly', 1), ('type', 1), ('place', 1), ('feeling', 1), ('lot', 1), ('anxiety', 1), ('already', 1), ('good', 1), ('start', 1), ('appointment', 1), ('well', 1), ('impressions', 1), ('front', 1), ('desk', 1), ('almost', 1), ('talking', 1), ('one', 1), ('little', 1), ('bit', 1), ('doctor', 1), ('except', 1), ('go', 1), ('taking', 1), ('ovaries', 1), ('hysterectomy', 1), ('big', 1), ('oversight', 1), ('think', 1)]
[('see', 4), ('would', 4), ('minutes', 3), ('someone', 3), ('office', 3), ('discuss', 2), ('cancer', 2), ('doctor', 2), ('patients', 2), ('us', 2), ('could', 2), ('without', 2), ('took', 1), ('mom', 1), ('aimee', 1), ('fleury', 1), ('endometrial', 1), ('waiting', 1), ('asked', 1), ('long', 1), ('staff', 1), ('annoying', 1), ('responded', 1), ('later', 1), ('rushes', 1), ('door', 1), ('skimming', 1), ('pages', 1), ('previous', 1), ('notes', 1), (

[('dentist', 2), ('gelareh', 1), ('amazing', 1), ('cleaning', 1), ('day', 1), ('discomfort', 1), ('teeth', 1), ('feel', 1), ('clean', 1), ('great', 1), ('manner', 1), ('hobbs', 1), ('highly', 1), ('professional', 1), ('spoke', 1), ('patiently', 1), ('confidently', 1), ('questions', 1), ('concerns', 1), ('plus', 1), ('pretty', 1), ('neat', 1), ('say', 1), ('tv', 1), ('definitely', 1), ('made', 1), ('trip', 1), ('enjoyable', 1), ('experience', 1)]
[('friendly', 1), ('staff', 1), ('professional', 1), ('knowledgeable', 1), ('pleasure', 1), ('dealt', 1), ('gelareh', 1), ('hygienist', 1), ('layla', 1), ('assistant', 1), ('paola', 1), ('receptionist', 1), ('hobbs', 1), ('individually', 1), ('everything', 1), ('one', 1), ('could', 1), ('expect', 1), ('specific', 1), ('roles', 1), ('together', 1), ('make', 1), ('trip', 1), ('dentist', 1), ('feel', 1), ('like', 1), ('visiting', 1), ('loving', 1), ('family', 1), ('rest', 1), ('assured', 1), ('need', 1), ('dental', 1), ('work', 1), ('done', 1), ('

In [110]:
dataset_A.head()

,comments,rating
0,many good things say place needed cleaning fil...,positive
1,found highly skilled experienced dentistry wil...,positive
2,even begin office incredibly good family love ...,positive
3,went toothache needed cleaning really friendly...,positive
4,found new dental office place amazing people w...,positive


In [80]:
preprocess_reviews(dataset_B)

In [81]:
dataset_B.head()

,comments,rating
0,arrived pm dealership closed thought left plen...,positive
1,dropped car wednesday morning diagnostic john ...,negative
2,parents buying cars donna dunnivan decade alwa...,positive
3,recently bought another car donna dunnivan coc...,positive
4,schedule appointment due airbag recall used we...,positive


In [82]:
dataset_A.groupby('rating')['rating'].count()

rating
negative     407
positive    1043
Name: rating, dtype: int64

In [ ]:
dataset_B.groupby('rating')['rating'].count()

In [116]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(dataset_A['comments'])
X.shape

(1450, 8525)